In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import os
from torchvision import datasets, transforms, models

In [3]:
train_transform=transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225]
        )
     ])

valid_transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
         [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225]
    )
])

In [4]:
train_dir = './train'
valid_dir = './valid'
test_dir  = './test'

train_data = datasets.ImageFolder(train_dir, transform=train_transform)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transform)


train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)



In [5]:
from torchvision.datasets.folder import default_loader

class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.file_paths = [os.path.join(root_dir, fname) 
                           for fname in os.listdir(root_dir) if fname.endswith('.jpg')]
        self.transform = transform
        self.loader = default_loader  # PIL image loader

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        image = self.loader(path)
        if self.transform:
            image = self.transform(image)
        return image, path  # return path if you want to track predictions

test_dataset = TestDataset('./test', transform=valid_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
model=models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad= False

num_classes=len(train_data.classes)
model.fc=nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = model.to(device)

c:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Dell/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 25.0MB/s]


Training the model

In [9]:
import torch

def train_model(model, train_loader, valid_loader, criterion, optimizer, device, epochs=10):
    best_acc = 0.0
    best_model_wts = None

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        # --- Training phase ---
        model.train()  # Set model to training mode (enables dropout, batchnorm updates)
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()        
            outputs = model(inputs)      
            loss = criterion(outputs, labels)  
            loss.backward()             
            optimizer.step()             

            
            running_loss += loss.item() * inputs.size(0) 
            _, preds = torch.max(outputs, 1)              
            running_corrects += torch.sum(preds == labels.data).item()
            total_samples += inputs.size(0)

        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects / total_samples
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        
        model.eval()  
        val_loss = 0.0
        val_corrects = 0
        val_samples = 0

        with torch.no_grad():  
            for inputs, labels in valid_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data).item()
                val_samples += inputs.size(0)

        val_epoch_loss = val_loss / val_samples
        val_epoch_acc = val_corrects / val_samples
        print(f"Val Loss: {val_epoch_loss:.4f} Acc: {val_epoch_acc:.4f}")

        # Save best model
        if val_epoch_acc > best_acc:
            best_acc = val_epoch_acc
            best_model_wts = model.state_dict()

    print(f"Best Validation Accuracy: {best_acc:.4f}")

    # Load best weights before returning
    model.load_state_dict(best_model_wts)
    return model


In [10]:
import torch.optim
criterion=nn.CrossEntropyLoss()
optimizer= torch.optim.AdamW(model.parameters(), lr=0.001)
trained_model = train_model(model, train_loader, valid_loader, criterion, optimizer, device, epochs=10)


Epoch 1/10
Train Loss: 0.4780 Acc: 0.8887
Val Loss: 0.2056 Acc: 0.9385
Epoch 2/10
Train Loss: 0.1935 Acc: 0.9410
Val Loss: 0.1510 Acc: 0.9536
Epoch 3/10
Train Loss: 0.1578 Acc: 0.9498
Val Loss: 0.1393 Acc: 0.9538
Epoch 4/10
Train Loss: 0.1410 Acc: 0.9539
Val Loss: 0.1282 Acc: 0.9578
Epoch 5/10
Train Loss: 0.1294 Acc: 0.9575
Val Loss: 0.1347 Acc: 0.9544
Epoch 6/10
Train Loss: 0.1242 Acc: 0.9585
Val Loss: 0.1103 Acc: 0.9624
Epoch 7/10
Train Loss: 0.1218 Acc: 0.9591
Val Loss: 0.1126 Acc: 0.9634
Epoch 8/10
Train Loss: 0.1160 Acc: 0.9615
Val Loss: 0.1172 Acc: 0.9615
Epoch 9/10
Train Loss: 0.1120 Acc: 0.9617
Val Loss: 0.1081 Acc: 0.9643
Epoch 10/10
Train Loss: 0.1085 Acc: 0.9634
Val Loss: 0.1152 Acc: 0.9613
Best Validation Accuracy: 0.9643


In [14]:
torch.save(model.state_dict(), "plant_disease_prediction_model.pth")


In [12]:
len(train_data.classes)

38

In [13]:
import json

# Save the class names used during training
with open("class_names.json", "w") as f:
    json.dump(train_data.classes, f)
